Link: https://datahack.analyticsvidhya.com/contest/kings-of-machine-learning/

# 1. Carga de Datos

## 1.1 Carga

In [1]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import mode, gmean, hmean

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.options.display.max_columns = 100

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [17]:
def rmse(act, pred ):
    return np.sqrt(np.mean(np.power(pred-act, 2)))

In [83]:
train1 = pd.read_csv("data/train1.csv")
train9 = pd.read_csv("data/train9.csv")
hero_data = pd.read_csv("data/hero_data.csv")
TARGET = 'kda_ratio'
train1.head()

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,1,41,1_41,201,125,3731.707317
1,2,17,2_17,427,234,2922.058381
2,5,106,5_106,240,130,5116.682739
3,6,65,6_65,206,122,4131.553861
4,7,69,7_69,697,382,2474.469306


In [84]:
test1 = pd.read_csv("data/test1.csv")
test9 = pd.read_csv("data/test9.csv")
TARGET = 'kda_ratio'
test1.head()

,user_id,hero_id,id,num_games
0,3,53,3_53,278
1,4,17,4_17,180
2,9,51,9_51,70
3,10,54,10_54,189
4,15,74,15_74,263


In [85]:
test9.head()

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,3,65,3_65,330,183,3418.470419
1,3,104,3_104,231,107,2522.639069
2,3,7,3_7,200,103,2793.255132
3,3,19,3_19,189,91,3493.495230
4,3,4,3_4,182,102,3586.402266


In [86]:
print(train1.shape, " ", train9.shape )
print(test9.shape, " ", test9.shape )

(2094, 6)   (18846, 6)
(8082, 6)   (8082, 6)


In [87]:
hero_data.head()

,hero_id,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate
0,1,agi,Melee,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31,22,22,12,1.3,2.8,1.8,150,0,1.45,310,0.5
1,2,str,Melee,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28,25,20,18,2.8,2.2,1.6,150,900,1.70,290,0.6
2,3,int,Ranged,Support:Disabler:Nuker:Durable,200,1.50,75,0.9,1.0,25,37,43,23,23,23,2.4,2.4,2.4,400,900,1.70,310,0.6
3,4,agi,Melee,Carry:Disabler:Jungler:Nuker:Initiator,200,1.50,75,0.9,0.0,25,33,39,23,24,18,2.7,3.0,1.7,150,900,1.70,285,0.5
4,5,int,Ranged,Support:Disabler:Nuker:Jungler,200,1.50,75,0.9,-1.0,25,19,25,16,16,16,2.0,1.6,2.9,600,900,1.70,275,0.5


In [88]:
train9 = train9.sort_values(by = ['user_id','num_games','hero_id'], ascending = [True, False, True]).reset_index(drop = True)
train_orders = pd.DataFrame(list(range(1,10))*len(train1),columns=['heroe_order'])
train9 = pd.concat([train9,train_orders], axis = 1)
train9.tail(12)

,user_id,hero_id,id,num_games,num_wins,kda_ratio,heroe_order
18834,2991,75,2991_75,146,82,3669.712794,7
18835,2991,69,2991_69,136,69,3596.651446,8
18836,2991,93,2991_93,133,73,4164.965986,9
18837,2992,76,2992_76,290,211,5471.003135,1
18838,2992,14,2992_14,162,91,3193.750000,2
18839,2992,17,2992_17,160,84,3831.970935,3
18840,2992,93,2992_93,137,83,4017.964072,4
18841,2992,11,2992_11,102,55,3661.654135,5
18842,2992,8,2992_8,97,59,4153.992395,6
18843,2992,34,2992_34,93,40,3571.428572,7


In [271]:
test9 = test9.sort_values(by = ['user_id','num_games','hero_id'], ascending = [True, False, True]).reset_index(drop = True)
test_orders = pd.DataFrame(list(range(1,10))*len(test1),columns=['heroe_order'])
test9 = pd.concat([test9,test_orders], axis = 1)
test9.tail(12)

,user_id,hero_id,id,num_games,num_wins,kda_ratio,heroe_order
8070,2987,8,2987_8,91,54,3393.088553,7
8071,2987,11,2987_11,78,44,2914.634146,8
8072,2987,22,2987_22,76,50,4653.758542,9
8073,2988,17,2988_17,298,178,3297.162767,1
8074,2988,1,2988_1,223,115,3290.661070,2
8075,2988,10,2988_10,170,111,3980.271270,3
8076,2988,59,2988_59,152,95,2249.566724,4
8077,2988,98,2988_98,152,87,3249.019608,5
8078,2988,34,2988_34,150,68,2807.453416,6
8079,2988,70,2988_70,149,70,2648.760331,7


## Grouping per User

In [189]:
f_wmean = lambda x:( (x.num_games * x.kda_ratio).sum()/ x.num_games.sum() )
f_sum = lambda x:( (x.num_games * x.kda_ratio).sum() )
f_kdaSum_gamesSum = lambda x:( x.kda_ratio.sum()/x.num_games.sum() )

### For Train

In [272]:
train9_temp = train9[(train9['heroe_order']>=2)&(train9['heroe_order']<=9)]

In [273]:
train_wmean10 = train9_temp.groupby(['user_id']).apply(f_wmean).reset_index().rename(columns ={0:'kda_ratio_wmean10'})

In [281]:
print(rmse(np.array(train1['kda_ratio']), np.array(train_wmean10['kda_ratio_wmean10'])))

766.270486653


In [259]:
list_a = list()
list_b = list()
list_rmse = list()
for a in range(1,10):
    for b in range(a,10):
        train9_temp = train9[(train9['heroe_order']>=a)&(train9['heroe_order']<=b)]
        train_wmean10 = train9_temp.groupby(['user_id']).apply(f_wmean).reset_index().rename(columns ={0:'kda_ratio_wmean10'})
        rmse_valor = rmse(np.array(train1['kda_ratio']), np.array(train_wmean10['kda_ratio_wmean10']))
        list_a.append(a)
        list_b.append(b)
        list_rmse.append(rmse_valor)

In [268]:
results_temp = pd.concat([pd.DataFrame(list_a,columns=['a']),
                          pd.DataFrame(list_b,columns=['b']),
                          pd.DataFrame(list_rmse,columns=['rmse'])],axis = 1)

In [270]:
results_temp.sort_values(by = ['rmse','a','b'], ascending = [True, True, True]).reset_index(drop = True)

,a,b,rmse
0,2,9,766.270487
1,4,9,766.517247
2,2,8,774.206655
3,3,9,776.117327
4,4,8,781.096544
5,2,7,785.294880
6,3,8,788.615512
7,2,6,795.451399
8,1,9,797.080678
9,4,7,801.513494


### For Test

In [300]:
test9_temp = test9[(test9['heroe_order']>=2)&(test9['heroe_order']<=9)]

In [301]:
test_wmean10 = test9_temp.groupby(['user_id']).apply(f_wmean).reset_index().rename(columns ={0:'kda_ratio_wmean10'})

In [302]:
subm = test_wmean10.rename(columns ={'kda_ratio_wmean10':'kda_ratio'})
subm.head()

,user_id,kda_ratio
0,3,3036.564348
1,4,3194.807998
2,9,3011.026531
3,10,3959.363168
4,15,4744.288995


In [303]:
subm = pd.merge(test1[['id','user_id']],subm, on = ['user_id'], how = 'left')
subm.head()

,id,user_id,kda_ratio
0,3_53,3,3036.564348
1,4_17,4,3194.807998
2,9_51,9,3011.026531
3,10_54,10,3959.363168
4,15_74,15,4744.288995


In [304]:
subm[['id','kda_ratio']].to_csv("subm/subm1_baseline.csv")

In [ ]:
1 - 766

In [298]:
train9_temp = train9[(train9['heroe_order']>=1)&(train9['heroe_order']<=9)]

In [299]:
train_mean10 = train9_temp.groupby(['user_id'])['kda_ratio'].mean().reset_index().rename(columns ={'kda_ratio':'kda_ratio_mean10'})
print(rmse(np.array(train1['kda_ratio']), np.array(train_mean10['kda_ratio_mean10'])))

761.666078125


In [33]:
print(rmse(np.array(train1['kda_ratio']), np.array(train_mean10['kda_ratio_mean10'])))

visitors_mean:
761.666078125


In [219]:
a = 2
b = 3

In [220]:
temp = (a*train_wmean10['kda_ratio_wmean10'] + b*train_mean10['kda_ratio_mean10'])/(a+b)
temp.head()

0    3878.507183
1    2860.897865
2    4183.317341
3    3877.734810
4    3005.878727
dtype: float64

In [221]:
print(rmse(np.array(train1['kda_ratio']), temp))

763.156925411


In [282]:
list_rmse = list()
for a in range(1,20):
    for b in range(1,20):
        temp = (a*train_wmean10['kda_ratio_wmean10'] + b*train_mean10['kda_ratio_mean10'])/(a+b)
        rmse_valor = rmse(np.array(train1['kda_ratio']), temp)
        list_rmse.append(rmse_valor)

In [283]:
min(list_rmse)

757.12115676090264

In [190]:
train9_temp = train9[train9['heroe_order']<10]

In [191]:
train_kdaSum_gamesSum10 = train9_temp.groupby(['user_id']).apply(f_kdaSum_gamesSum).reset_index().rename(
                                                                                    columns={0:'kda_ratio_kdaSum_gamesSum10'})

In [192]:
train1_10 = pd.merge(train1,train_kdaSum_gamesSum10, on = ['user_id'], how = 'left')
train1_10['kda_ratio_k10'] = train1_10['num_games']*train1_10['kda_ratio_kdaSum_gamesSum10']

print(rmse(np.array(train1['kda_ratio']), np.array(train1_10['kda_ratio_k10'])))

1249.09345476


In [144]:
print(rmse(np.array(train1['kda_ratio']), np.array(train1_10['kda_ratio_k10'])))

1249.09345476
